In [10]:
from pruners.pruning_methods import Pruner, PruningMethod
from pruners.Pruner_Test_Suite import *
from pruners.Fine_Tuning import *
import torch
from evaluations.evaluator import Evaluator
device = torch.device('mps')

In [15]:
from transformers import AutoTokenizer, BertForSequenceClassification
from explainers.explanation_methods import SHAP, LIME
import shap
import torch
from copy import deepcopy

tokenizer = AutoTokenizer.from_pretrained(
    "textattack/bert-base-uncased-yelp-polarity")
model = BertForSequenceClassification.from_pretrained(
    "textattack/bert-base-uncased-yelp-polarity")
print('model loaded')

input = 'Hello, my dog is so terribly ugly'
tokenized_input = tokenizer(input, return_tensors="pt")
label = torch.tensor([0]).unsqueeze(0)  # Batch size 1

pruned_model = deepcopy(model)
unstructured_model = deepcopy(model)

l1_paras_to_prune = []
random_paras_to_prune = []


for layer in pruned_model.bert.encoder.layer:
    # Attention weights (query, key, value, and output projection)
    l1_paras_to_prune.append((layer.attention.self.query, 'weight'))
    l1_paras_to_prune.append((layer.attention.self.key, 'weight'))
    l1_paras_to_prune.append((layer.attention.self.value, 'weight'))
    l1_paras_to_prune.append((layer.attention.output.dense, 'weight'))
    
    # Intermediate dense layer
    l1_paras_to_prune.append((layer.intermediate.dense, 'weight'))
    
    # Output dense layer
    l1_paras_to_prune.append((layer.output.dense, 'weight'))

for layer in unstructured_model.bert.encoder.layer:
    # Attention weights (query, key, value, and output projection)
    random_paras_to_prune.append((layer.attention.self.query, 'weight'))
    random_paras_to_prune.append((layer.attention.self.key, 'weight'))
    random_paras_to_prune.append((layer.attention.self.value, 'weight'))
    random_paras_to_prune.append((layer.attention.output.dense, 'weight'))
    
    # Intermediate dense layer
    random_paras_to_prune.append((layer.intermediate.dense, 'weight'))
    
    # Output dense layer
    random_paras_to_prune.append((layer.output.dense, 'weight'))

# Initialize Pruner instance
Bert_pruner = Pruner(pruned_model)
Unstructured_pruner = Pruner(unstructured_model)

# Initialize PruningMethod instances
L1_prune_Bert = PruningMethod(type = "L1Unstructured", paras_to_prune = l1_paras_to_prune, percentage = 0.2, mask = None)
Unstructured_prune = PruningMethod(type='RandomUnstructured', paras_to_prune= random_paras_to_prune, percentage=0.2, mask=None)

# Prune the model
Bert_pruner.prune(L1_prune_Bert)
L1_prune_Bert.remover()

Unstructured_pruner.prune(Unstructured_prune)
Unstructured_prune.remover()

pruned_shapExplainer = SHAP(pruned_model, tokenizer, device=device)
unstructured_shapExplainer = SHAP(unstructured_model, tokenizer, device=device)
normal_shapExplainer = SHAP(model, tokenizer, device=device)

# pruned_explanation = pruned_shapExplainer.explain(input)
# normal_explanation = normal_shapExplainer.explain(input)

unstructured_evaluator = Evaluator(unstructured_shapExplainer)
pruned_evaluator = Evaluator(pruned_shapExplainer)
normal_evaluator = Evaluator(normal_shapExplainer)

inputs = ['Camilo CANNOT CODE FOR HIS LIFE. I DONT LIKE HIM!!!!',
          'David is GREAT at soccer. Can recommend <thumbs up>!',
          'Joey is joey. I feel very neutrally about him',
          'Finale is the best professor Harvard has EVER had. Would recommend!',
          'I AM GOING TO SCREAMMMMMMMMMMM AHHHHHHHHHHHH',
          'Paula and Hiwot are great TFs!']

for input in inputs:
    print(f'Evaluated on {input}')
    print(f'Normal infedility: {normal_evaluator.get_local_infidelity(input):4f}')
    print(f'Unstructured Pruning Infidelity: {unstructured_evaluator.get_local_infidelity(input):4f}')
    print(f'L1 Pruned infidelity: {pruned_evaluator.get_local_infidelity(input):4f}')
    print('*' * 80)

model loaded
Evaluated on Camilo CANNOT CODE FOR HIS LIFE. I DONT LIKE HIM!!!!


Normal infedility: 16.697126


Unstructured Pruning Infidelity: 0.667052


L1 Pruned infidelity: 12.824641
********************************************************************************
Evaluated on David is GREAT at soccer. Can recommend <thumbs up>!
Normal infedility: 0.066186
Unstructured Pruning Infidelity: 0.984423
L1 Pruned infidelity: 0.044637
********************************************************************************
Evaluated on Joey is joey. I feel very neutrally about him
Normal infedility: 0.476829
Unstructured Pruning Infidelity: 0.529545
L1 Pruned infidelity: 1.966267
********************************************************************************
Evaluated on Finale is the best professor Harvard has EVER had. Would recommend!
Normal infedility: 0.152795
Unstructured Pruning Infidelity: 0.455202
L1 Pruned infidelity: 0.149887
********************************************************************************
Evaluated on I AM GOING TO SCREAMMMMMMMMMMM AHHHHHHHHHHHH
Normal infedility: 25.010798
Unstructured Pruning Infidelity: 0.009562
L1 Pru